In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

In [2]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 128, kernel_size=7),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 128, kernel_size=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, kernel_size=4),
            nn.ReLU(inplace=True)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(256*6*6, 4096),
            nn.Sigmoid(),
            nn.Linear(4096, 1)
        )
        
    def forward_one(self, x):
        x = self.cnn(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x
    
    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [3]:
# Example dataset class
class SeekerDonatorDataset(Dataset):
    def __init__(self, seekers, donators, labels, transform=None):
        self.seekers = seekers
        self.donators = donators
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        seeker = self.seekers[idx]
        donator = self.donators[idx]
        label = self.labels[idx]

        if self.transform:
            seeker = self.transform(seeker)
            donator = self.transform(donator)

        return seeker, donator, label

In [ ]:
# Example usage
transform = transforms.Compose([transforms.ToTensor()])
seekers = []  # Load your seeker data
donators = []  # Load your donator data
labels = []  # Load your labels

dataset = SeekerDonatorDataset(seekers, donators, labels, transform=transform)
dataloader = DataLoader(dataset, shuffle=True, batch_size=32)

model = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    for i, data in enumerate(dataloader, 0):
        seeker, donator, label = data
        optimizer.zero_grad()
        output1, output2 = model(seeker, donator)
        loss = criterion(output1, output2, label)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f"Epoch {epoch}, Batch {i}, Loss {loss.item()}")

TypeError: object of type 'ellipsis' has no len()